<a href="https://colab.research.google.com/github/rodolv1979/PyTorch_experiment_Image_classificator/blob/main/py_torch_course_o_reilly_EXAMPLE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Example 1**
First neural network

Select Runtime as GPU

In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# **Define the model**

This model has three layers (500, 1000, 10). The last one is important, since we have 10 classes we need 10 output neurons

In [4]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 500) # Please take a look seriously to those values in -> 784 and out -> 500
    self.fc2 = nn.Linear(500, 1000) # in -> 500 and out -> 1000
    self.fc3 = nn.Linear(1000, 10) # in -> 1000 and out 10

  # This defines the forward function
  def forward(self, x):
    x = x.view(-1, 784) # -1 is the fitting of the input function
    x = F.relu(self.fc1(x)) # Calculate the output of the first layer
    x = F.relu(self.fc2(x)) # Calculate the output of the second layer
    x = self.fc3(x) # # Calculate the output of the network
    return F.log_softmax(x, dim=1) # Logaritmic soft_max function

# **Loading Data**

Using DataLoader and DataSet torch util classes. 

In [7]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, 
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                                 ])),
    batch_size=128, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                   transform=transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))                                                                         
    ])),
     batch_size=1000, shuffle=True)


# **Training and testing loops**

Boilerplate code for training and testing (almost never changes).

In [9]:
def train(model, device, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0:
      print('Train Epoch: {} [{}, {} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
      pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()
  
  test_loss /= len(test_loader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
